# Laboratoire 3 - Caméra
Matériel fourni: règle en aluminium de 1 m, règle en plastique de 30 cm, rapporteur d'angle, repère vertical.

## Partie 0 - Interagir avec la caméra

D'abord, exécutons un peu de code de configuration, puis connectons-nous au robot.

In [ ]:
%pylab inline
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
mpl.rc('animation', html='html5') #display animated plots inline

from robmob.robot import Robot
from robmob.sensors import KinectRGBSensor
from robmob.visualization import Visualizer

In [ ]:
ip_robot = '192.168.0.100'
robot = Robot(ip_robot)
robot.connect()

Le code suivant sert à créer ajouter la Kinect aux capteurs du robot.

In [ ]:
kinect = KinectRGBSensor()
robot.add_sensor(kinect)

> **PROTIP** Aussitôt que vous exécutez la ligne `robot.add_sensor(kinect)`, le flux de la caméra est transmi 
> par le réseau du robot jusqu'à votre ordinateur. Comme ces données causent un bon traffic sur le routeur, 
> tentez de ne pas vous connecter à ce flux de données plusieurs fois en parallèle!

## Acquisition d'images

Pour afficher la dernière image capturée par la Kinect, on peut utiliser `peek_data`.

In [ ]:
kinect.peek_data()

Même si ce n'est pas tout à fait nécessaire dans le cadre de ce laboratoire, on peut aussi afficher les images de la kinect en continu avec le code suivant.

In [ ]:
from IPython.display import display,HTML,clear_output
import time

N_OF_FRAMES_TO_DISPLAY = 100

for i in range(N_OF_FRAMES_TO_DISPLAY):
    clear_output(wait=True)
    display(kinect.peek_data())
    time.sleep(0.03333)

## Partie 1 - Calibration de la longueur focale

L'objectif de cette partie de de déterminer la longeur focale de la caméra du robot. Nous utiliserons la caméra RGB de la Kinect, en ignorant pour l'instant la partie infra-rouge des données.

Pour commencer, placez la règle de 1 m sur une table, puis placez le robot à une des extrémités de la règle. À l'autre extrémité, placez la règle de plastique de 30 cm à la même hauteur que la caméra, de sorte que la Kinect observe directement la règle de plastique. Votre assemblage devrait ressembler à l'image ci-bas.

![Assemblage pour la calibration](img/assemblage_calibration.jpg)

In [ ]:
ginput()